In [1]:
import numpy as np

In [2]:
import tensorflow as tf

In [3]:
import torch

In [4]:
import transformers

In [5]:
from transformers import pipeline

In [ ]:
classifier=pipeline("text-classification")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [11]:
from transformers import DistilBertModel, DistilBertTokenizer

model = DistilBertModel.from_pretrained('distilbert-base-uncased')
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

C:\Users\Rambabu\AppData\Roaming\Python\Python310\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Rambabu\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
C:\Users\Rambabu\AppData\Roaming\Python\Python310\site-packages\torch\storage.py:233: UserWarning: Failed to initialize NumPy:

In [13]:
from transformers import RobertaModel, RobertaTokenizer

model = RobertaModel.from_pretrained('roberta-base')
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
import torch.nn.functional as F

In [7]:
from transformers import AutoTokenizer

In [8]:
from tensorflow.keras.layers import Embedding,Dense,LayerNormalization,Dropout,MultiHeadAttention

In [9]:
from torch.nn import Linear,Embedding,LayerNorm,Dropout,Module,MultiheadAttention

In [356]:
def tf_scaled_dot_product(query,key,value,mask=None):
    dk=tf.shape(query)[-1]
    key=tf.transpose(key,perm=(0,1,3,2))
    score=tf.matmul(query,key)/np.sqrt(dk)
    if mask is not None:
        score+=mask
    attention_wts=tf.nn.softmax(score,axis=-1)
    context_vector=tf.matmul(attention_wts,value)
    return context_vector,score

In [361]:
p=np.random.random((1,8,4,64))

In [362]:
tfa,m=tf_scaled_dot_product(p,p,p)

In [363]:
tfa.shape

TensorShape([1, 8, 4, 64])

In [49]:
import math

In [85]:
def torch_scaled_dot_product(query,key,value,mask=None):
    dk=query.size()[-1]
    key=key.transpose(-1,-2)
    score=query.matmul(key)/np.sqrt(dk)
    if mask is not None:
        score+=mask
    attention_wts=F.softmax(score,dim=-1)
    context_vector=torch.matmul(attention_wts,value)
    return score,context_vector

In [136]:
q=torch.rand((1,8,4,64))

In [137]:
q.size()

torch.Size([1, 8, 4, 64])

In [138]:
score,ta=torch_scaled_dot_product(q,q,q)

In [139]:
ta.shape

torch.Size([1, 8, 4, 64])

In [140]:
score.size(),score.shape

(torch.Size([1, 8, 4, 4]), torch.Size([1, 8, 4, 4]))

In [297]:
mask=torch.full(score.size(),float('-inf'))

In [285]:
mask0=torch.triu(mask,1)

In [286]:
mask0[0][0]

tensor([[0., -inf, -inf, -inf],
        [0., 0., -inf, -inf],
        [0., 0., 0., -inf],
        [0., 0., 0., 0.]])

In [300]:
mask2=torch.tril(mask,diagonal=-1)

In [301]:
mask2[0][0]

tensor([[0., 0., 0., 0.],
        [-inf, 0., 0., 0.],
        [-inf, -inf, 0., 0.],
        [-inf, -inf, -inf, 0.]])

In [370]:
mask1=tf.fill(score.size(),float('-inf'))

In [373]:
mask1=tf.linalg.band_part(mask1,0,-1)

In [371]:
mask1[0][0]

<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
array([[-inf, -inf, -inf, -inf],
       [-inf, -inf, -inf, -inf],
       [-inf, -inf, -inf, -inf],
       [-inf, -inf, -inf, -inf]], dtype=float32)>

In [351]:
mask=torch.tril(torch.ones(score.size()),0)

In [352]:
mask.size()

torch.Size([1, 8, 4, 4])

In [364]:
m.shape

TensorShape([1, 8, 4, 4])

In [390]:
scores1=tf.where(mask1!=0,float('-inf'),)

InvalidArgumentError: {{function_node __wrapped__SelectV2_device_/job:localhost/replica:0/task:0/device:CPU:0}} condition [1,8,4,4], then [], and else [1,8,4,1,4] must be broadcastable [Op:SelectV2] name: 

In [387]:
scores1[0][0]

<tf.Tensor: shape=(4, 4), dtype=float64, numpy=
array([[      -inf,       -inf,       -inf,       -inf],
       [1.84327783,       -inf,       -inf,       -inf],
       [1.97818566, 1.87336725,       -inf,       -inf],
       [2.01660168, 1.82149627, 2.00311921,       -inf]])>

In [389]:
scores1[0][0][0]

<tf.Tensor: shape=(4, 4), dtype=float64, numpy=
array([[      -inf,       -inf,       -inf,       -inf],
       [1.84327783,       -inf,       -inf,       -inf],
       [1.97818566, 1.87336725,       -inf,       -inf],
       [2.01660168, 1.82149627, 2.00311921,       -inf]])>

In [376]:
scores=score.masked_fill(mask==0,float('-inf'))

In [377]:
scores[0][0]

tensor([[2.7567,   -inf,   -inf,   -inf],
        [2.2605, 2.9969,   -inf,   -inf],
        [1.9832, 2.0995, 2.5692,   -inf],
        [1.9208, 1.9373, 1.7918, 2.3374]])